# LLM-Based Chatbot for Google Play Apps

I developed an intelligent NLP chatbot capable of answering questions about Google Play applications using a RAG (Retrieval-Augmented Generation) pipeline. The system integrates natural language processing (NLP) with Pandas for data handling, FAISS for semantic search via embeddings, and a LLM to generate responses in natural language. Key functionalities include:

- App ranking by price, rating, or popularity.
- Insight generation on user behavior and trends.

Technologies: Python, NLP, Pandas, LangChain, FAISS, RAG, LLMs (LLaMA, Gemma:2b, Phi3:mini), Google Play Store Dataset.

---

## Library Import

In [4]:
from IPython.utils import io
with io.capture_output():
    !pip install -q -U sentence-transformers
    !pip install -q tf-keras
    !pip install -q langchain sentence-transformers faiss-cpu openai
    !pip install -q sentence-transformers faiss-cpu pandas
    !pip install -q langchain faiss-cpu sentence-transformers transformers accelerate
    !pip install -q langchain faiss-cpu sentence-transformers
    !pip install -q huggingface_hub
    !pip install -q ollama

In [6]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from huggingface_hub import hf_hub_download
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('googleplaystore.csv')

In [8]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


---

## Data Preparation

In [11]:
print("Number of Rows:", df.shape[0])
print("Number of Columns:", df.shape[1])

Number of Rows: 10841
Number of Columns: 13


In [12]:
# Columns and Data Type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


---

Variable change **Last Update** from object type to **datetime** with year-month-day format, NaT for invalid values.

The **Last update** variable is subsequently changed to a **string** so that NLP can better interpret the dates.

In [15]:
#df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df['Last Updated'] = pd.to_datetime(df['Last Updated'], format="%B %d, %Y", errors='coerce')

In [16]:
df['Last Updated'] = df['Last Updated'].astype(str)

---

Handling **null** or **missing** values

In [19]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [20]:
# Display all records with NaN values in the data frame
df_null = df[df.isnull().any(axis = 1)]
df_null

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
15,Learn To Draw Kawaii Characters,ART_AND_DESIGN,3.2,55,2.7M,"5,000+",Free,0,Everyone,Art & Design,2018-06-06,NaN,4.2 and up
23,Mcqueen Coloring pages,ART_AND_DESIGN,NaN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,2018-03-07,1.0.0,4.1 and up
113,Wrinkles and rejuvenation,BEAUTY,NaN,182,5.7M,"100,000+",Free,0,Everyone 10+,Beauty,2017-09-20,8.0,3.0 and up
123,Manicure - nail design,BEAUTY,NaN,119,3.7M,"50,000+",Free,0,Everyone,Beauty,2018-07-23,1.3,4.1 and up
126,Skin Care and Natural Beauty,BEAUTY,NaN,654,7.4M,"100,000+",Free,0,Teen,Beauty,2018-07-17,1.15,4.1 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10824,Cardio-FR,MEDICAL,NaN,67,82M,"10,000+",Free,0,Everyone,Medical,2018-07-31,2.2.2,4.4 and up
10825,Naruto & Boruto FR,SOCIAL,NaN,7,7.7M,100+,Free,0,Teen,Social,2018-02-02,1.0,4.0 and up
10831,payermonstationnement.fr,MAPS_AND_NAVIGATION,NaN,38,9.8M,"5,000+",Free,0,Everyone,Maps & Navigation,2018-06-13,2.0.148.0,4.0 and up
10835,FR Forms,BUSINESS,NaN,0,9.6M,10+,Free,0,Everyone,Business,2016-09-29,1.1.5,4.0 and up


There are **10,841** records in the dataframe, so the **1,474** records with **NaN** values are **removed**, as they do not represent a significant amount for the dataframe.

In [22]:
df.dropna(subset = ['Rating','Content Rating', 'Current Ver', 'Android Ver', 'Last Updated'], axis = 0, inplace = True)

In [23]:
df.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [35]:
# Number of records after removing NaN values
df.shape[0]

9360

---

Conversion of key **categorical variables** for our ML model to **numerical variables**

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             9360 non-null   object 
 1   Category        9360 non-null   object 
 2   Rating          9360 non-null   float64
 3   Reviews         9360 non-null   object 
 4   Size            9360 non-null   object 
 5   Installs        9360 non-null   object 
 6   Type            9360 non-null   object 
 7   Price           9360 non-null   object 
 8   Content Rating  9360 non-null   object 
 9   Genres          9360 non-null   object 
 10  Last Updated    9360 non-null   object 
 11  Current Ver     9360 non-null   object 
 12  Android Ver     9360 non-null   object 
dtypes: float64(1), object(12)
memory usage: 1023.8+ KB


In [61]:
df.sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
5345,Quran for Android,BOOKS_AND_REFERENCE,4.7,497826,Varies with device,"10,000,000+",Free,0,Everyone,Books & Reference,"May 30, 2018",Varies with device,Varies with device
2248,Paramedic Protocol Provider,MEDICAL,4.5,171,20M,"10,000+",Paid,$10.00,Everyone 10+,Medical,"September 21, 2017",1.8.3,4.1 and up
2011,Farm Heroes Saga,GAME,4.4,7614407,70M,"100,000,000+",Free,0,Everyone,Casual,"July 26, 2018",5.1.8,2.3 and up
9148,Command & Conquer: Rivals,FAMILY,NaN,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device
5168,Downers Grove AH,MEDICAL,NaN,0,29M,100+,Free,0,Everyone,Medical,"January 10, 2018",300000.0.96,4.0.3 and up


The **Price** variable is formatted, **removing “$”** and **spaces**. It is converted **from object type to float64 type**.

In [42]:
df['Price'] = df['Price'].str.replace('$', '', regex = False).astype(float)

The **Reviews** variable is converted **from type object to type float64**. 

In [45]:
df['Reviews'] = pd.to_numeric(df['Reviews'], errors = 'coerce')

For the **Size** variable, the values are in **“Megabytes,” “Kilobytes,” and “Varies with device.”**
First, a **standard** will be established so that **all apps have their Size in Mb**, **dividing by 1024** for Kb cases.
For **“Varies with device”** cases, the value is set to **NaN**.

In [48]:
# Size
def conv_size(value):
    if 'M' in value:
        return float(value.replace('M', ''))
    elif 'k' in value:
        return float(value.replace('k', '')) / 1024
    elif 'Varies with device' in value:
        return None
    return None
        
df['Size_num'] = df['Size'].apply(conv_size)

In [50]:
df['Size_num'].isnull().sum()

1637

The number of cases of **“Varies with device” - NaN** is **1637**.
The values are **replaced** with the **average** based on the **app Category** to which they **belong**.

In [53]:
df_size_NaN = df[df['Size_num'].isnull()]
category_count =  df_size_NaN['Category'].value_counts()
# Category name
category_count.index

for c in category_count.index:
    category_mean = df[df['Category'] == c]['Size_num'].mean()
    df.loc[(df['Category'] == c) & (df['Size_num'].isnull()), 'Size_num'] = category_mean

For the **Installs** variable of type object, the field is formatted by **replacing “+” and “,”** with **spaces** and **converting** it to type **int**.
The **column name** is changed to **Installs+** to refer to the previous symbology.

In [56]:
df['Installs'] = df['Installs'].str.replace('+', '', regex = False).str.replace(',', '', regex = False)

df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce').astype('Int64')
df.rename(columns = {'Installs' : 'Installs+'}, inplace = True )

In [58]:
df.head()

,App,Category,Rating,Reviews,Size,Installs+,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_num
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,19.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,14.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,8.7
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,25.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,2.8


---

## Model Development

The system integrates LLaMA with a RAG pipeline, combining FAISS vector search and LLM generation. Queries are transformed into embeddings and matched against vectorized app data, enabling LLaMA to generate context-aware answers grounded in the retrieved information.

### <font color = 'blue'> LM-Based System: LLaMA + FAISS + RAG Pipeline (Retrieval-Augmented Generation) </font>

A **document** is created by grouping **“text”** and **“metadata”**.
- Text: main content used to generate responses.
- Metadata: contextual information.

This allows for:
- Easier retrieval: Search engines such as vectors use text.
- Providing additional context: Metadata can be used to filter or classify results.
- Optimize performance: Separate what needs to be indexed from what is only for display or filtering.

This is a common and useful practice for **semantic search** and **RAG** systems.

In [65]:
#1. Create the documents
from langchain.schema import Document

documents = []
for _, row in df.iterrows():
    text = (
        f"App: {row['App']}. "
        f"Category: {row['Category']}. "
        f"Genres: {row['Genres']}. "
        f"Price: {row['Price']}. "
        f"Rating: {row['Rating']}. "
        f"Size: {row['Size_num']}. "
        f"Reviews: {row['Reviews']}. "
        f"Current Ver: {row['Current Ver']}. "
        f"Android Ver: {row['Android Ver']}."
    )
    metadata = {
        "Type": row['Type'],
        "Last Updated": row['Last Updated'],
        "Content Rating": row['Content Rating'],
        "Installs+": row['Installs+']
    }
    documents.append(Document(page_content = text, metadata = metadata))

In [67]:
#2. Definition and loading of the embedding model
from langchain_community.embeddings import HuggingFaceEmbeddings

# Embeddings model
hf_embed = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2") # lightweight model (MiniLM) trained to convert text into vectors

The **texts** are **converted** into numerical **vectors** or **embeddings** to then perform a **FAISS library similarity search**. 

**Similarity** is **measured** using metrics such as **Cosine Similarity** or **Euclidean Distance**.

**FAISS** returns the documents whose **vectors** are **closest** to the query vector.

In [69]:
#3. Create the vector store with FAISS for similarity search
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embedding = hf_embed)

In [70]:
#4. Define and load the local LLM model Ollama with LangChain framework
from langchain_community.llms import Ollama

llm = Ollama(model="gemma:2b")
#llm = Ollama(model="llama2")
#llm = Ollama(model="phi3:mini")

1. Search the documents.
2. Use the LLM to generate a response.
3. Return the answer and the sources used.

In [77]:
#5. Build the RAG (Retrieval-Augmented Generation) pipeline
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,                                    # model
    retriever = vectorstore.as_retriever(),       # search for the most relevant documents in your vector database
    return_source_documents = True                # In addition to the answer also return the documents used as sources
)

A **hybrid scheme** was implemented for the chatbot to work with different types of questions.

LLaMA **LLM model** to interpret and generate a response in **natural language**.

**Filters and validations** for **numerical calculations** with **Pandas**.

**LLM models and FAISS** similarity search **are not reliable** with **numerical operations**, which is why hybrid schemes are very applicable in real production environments.

For the Google PlayStore review chatbot, I performed different types of validations on the possible questions that users might ask in relation to **numerical calculations**, involving keywords such as: **“Top,” “expensive,” “cheap,” “size,” etc**, also covering words in Spanish , trying to cover as many possibilities as possible.

If the question asked by the user does not contain the defined keywords, the response will be generated with a **semantic response** using the LLM and FAISS models.

In [158]:
# Question Type
import re

def question_type(question: str):
    question = question.lower()
    num = None

    # Extract number for answers with "Top"
    num_match = re.search(r'\d+', question)
    if num_match:
        num = int(num_match.group())

    # TOP queries
    if "top" in question:
        if "caro" in question or "caras" in question or "expensive" in question:
            return "top_max_price", num
        elif "barata" in question or "baratas" in question or "cheap" in question or "cheapest" in question:
            return "top_min_price", num
        elif "rating" in question:
            if "mejor" in question or "mas" in question or "mayor" in question or "best" in question:
                return "top_max_rating", num
            elif "peor" in question or "menos" in question or "menor" in question or "worst" in question:
                return "top_min_rating", num
        else:
            return "top_unspecified", num

    # Numeric Queries
    if "caro" in question or "cara" in question or "expensive" in question:
        return "max_price", None
    if "barata" in question or "barato" in question or "cheap" in question or "cheapest" in question:
        return "min_price", None
    if "ratings" in question or "rating" in question:
        if "mejor" in question or 'mayor' in question or "best" in question or "most" in questions or 'more' in question or 'highest' in question:
            return "max_rating", None
        elif "peor" in question or 'menor' in question or "worst" in question or 'less' in questions or 'lowest ' in question:
            return "min_rating", None
    if "reviews" in question or 'review' in question:
        if "mas" in question or "mayor" in question or "more" in question or 'most' in question or 'highest' in question:
            return "rev_more", None
        elif "menos" in question or "menor" in question or "less" in question or 'lowest ' in question:
            return "rev_less", None
    if "tamaño" in question or "pesada" in question or "size" in question:
        if "mas" in question or "mayor" in question or "more" in question:
            return "size_more", None
        elif "menos" in question or "menor" in question or "less" in question:
            return "size_less", None

    # Default Semantic Query
    return "semantic", None

The type of question asked by the user is processed based on the text validations from the previous step. Validations and filters are executed in Pandas for questions related to numerical operations, or the LLM + FAISS model is executed for semantic questions.

The model **prompt** is defined to:
- Understand the context
- Know what type of output is expected
- Activate its generation mechanism

A **prompt is necessary**, even if the user does not always see it directly. It is the **essential input for an LLM to function**.

In [160]:
def answer(query: str):
    # Question type detection
    tipo, num = question_type(query)
    #print(tipo, num)

    # Type of response according to the type
    if tipo == "max_price":
        app = df.loc[df['Price'].idxmax()]
        return f"The most expensive app is *{app['App']}* with a price of ${app['Price']:.2f}."

    elif tipo == "min_price":
        app = df.loc[df['Price'].idxmin()]
        return f"The cheapest app is *{app['App']}* with a price of ${app['Price']:.2f}."

    elif tipo == "top_max_price" and num:
        top_apps = df.sort_values("Price", ascending=False).head(num)
        return "Top {} most expensive apps: {}".format(
            num,
            "\n".join([f"- {row['App']} (${row['Price']:.2f})" for _, row in top_apps.iterrows()])
        )

    elif tipo == "top_min_price" and num:
        top_apps = df.sort_values("Price").head(num)
        return "Top {} cheaper apps: {}".format(
            num,
            "\n".join([f"- {row['App']} (${row['Price']:.2f})" for _, row in top_apps.iterrows()])
        )

    elif tipo == "max_rating":
        app = df.loc[df['Rating'].idxmax()]
        return f"The highest-rated app is *{app['App']}* with Rating {app['Rating']}."

    elif tipo == "min_rating":
        app = df.loc[df['Rating'].idxmin()]
        return f"The app with the worst rating is *{app['App']}* with Rating {app['Rating']}."

    elif tipo == "top_max_rating" and num:
        top_apps = df.sort_values("Rating", ascending=False).head(num)
        return "Top {} highest-rated apps: {}".format(
            num,
            "\n".join([f"- {row['App']} ({row['Rating']})" for _, row in top_apps.iterrows()])
        )

    elif tipo == "top_min_rating" and num:
        top_apps = df.sort_values("Rating").head(num)
        return "Top {} apps with the worst ratings: {}".format(
            num,
            "\n".join([f"- {row['App']} ({row['Rating']})" for _, row in top_apps.iterrows()])
        )

    elif tipo == "rev_more":
        app = df.loc[df['Reviews'].idxmax()]
        return f"The app with the most reviews is *{app['App']}* with {app['Reviews']} reviews."

    elif tipo == "rev_less":
        app = df.loc[df['Reviews'].idxmin()]
        return f"The app with the fewest reviews is *{app['App']}* with {app['Reviews']} reviews."

    elif tipo == "size_more":
        app = df.loc[df['Size_num'].idxmax()]
        return f"The heaviest app is *{app['App']}* with {app['Size_num']:.2f} MB."

    elif tipo == "size_less":
        app = df.loc[df['Size_num'].idxmin()]
        return f"The lightest app is *{app['App']}* with {app['Size_num']:.2f} MB."

    # Semantic Answer (FAISS + LLM)
    elif tipo == "semantic":
        docs = vectorstore.similarity_search(query, k = 30)
        context = "\n".join([d.page_content for d in docs])

        prompt = (
            "You are an expert assistant specialized in Google Play apps"
            "Based on the following context, provide a clear and detailed answer "
            "to the user's question."
            f"Context:{context}"
            f"Question: {query}"
            "Answer in English:"
        )

        final_answer = llm.invoke(prompt)  # model: gemma:2b
        return final_answer

    else:
        return "I didn't understand your question. Please rephrase it."

In [164]:
questions = ["What is the most expensive app in the ENTERTAINMENT category?",
            "What are the top 5 apps with the best ratings in the BOOKS_AND_REFERENCE category?",
            "Which app has the most reviews in SOCIAL category?",
            "The highest-rated educational app according to your criteria",
            "Which shoping app would you recommend for users who want high ratings and lots of reviews, and why?"]

answers = [answer(q) for q in questions]

In [166]:
pd.set_option('display.max_colwidth', None)

response = {'Question': questions, 
            'Answer': answers}
df_results = pd.DataFrame(response)
df_results

,Question,Answer
0,What is the most expensive app in the ENTERTAINMENT category?,The most expensive app is *I'm Rich - Trump Edition* with a price of $400.00.
1,What are the top 5 apps with the best ratings in the BOOKS_AND_REFERENCE category?,"Top 5 highest-rated apps: - Mindvalley U Tallinn 2018 (5.0)\n- Ek Vote (5.0)\n- ei Calc (5.0)\n- Startupticker.ch News, Events (5.0)\n- Kolkata News:Anandbazar Patrika,ei samay&AllRating (5.0)"
2,Which app has the most reviews in SOCIAL category?,The app with the most reviews is *Facebook* with 78158306 reviews.
3,The highest-rated educational app according to your criteria,"Sure, based on the criteria you provided, **Khan Academy** emerges as the highest-rated educational app.\n\n**Category:** EDUCATION\n**Genres:** Education\n**Price:** Free\n**Rating:** 4.6/5\n**Size:** 21.0\n**Reviews:** 85375\n**Current Ver:** 5.0.0\n\nKhan Academy offers a wide range of educational content, including videos, exercises, and practice questions, across various subjects. It is widely used by students of all ages and is considered one of the most trusted educational platforms globally."
4,"Which shoping app would you recommend for users who want high ratings and lots of reviews, and why?","Based on the context, **Shopfully - Weekly Ads & Deals** would be the shoping app that would be highly recommended for users who want high ratings and lots of reviews. It has a rating of 4.4 with 279,428 reviews, and a positive overall sentiment, which suggests that users have had a positive experience with the app."


It should be noted that due to the magnitude of the LLM used, which belongs to the all-MiniLM-L6-v2 category, it is a very lightweight model. It is expected that the chatbot will not respond to very specific or complex reasoning questions or questions that are not found as parameters within the document.

---

## Conclusion

This project implements a Google Play reviews chatbot using a Retrieval-Augmented Generation (RAG) pipeline, combining semantic embeddings (FAISS + Hugging Face) with an LLM (LLaMA, Gemma:2b, Phi3:mini). Through carefully designed prompts, the system answers both open-ended natural language queries and numerical-driven questions (ranking, pricing, ratings), achieving a hybrid approach aligned with real-world NLP and modern chatbot applications.